In [1]:
# read data
data=read.csv("Remission.csv")

In [2]:
head(data)

Remiss,Cell,Smear,Infil,Li,Blast,Temp
1,-8,-83,-66,1.9,1.100,-996
1,-9,-36,-32,1.4,-74.000,-992
0,-8,-88,-7,-8.0,-176.000,-982
0,1,-87,-87,-7.0,1.053,-986
1,-9,-75,-68,1.3,-519.000,-98
0,1,-65,-65,-6.0,-519.000,-982


In [3]:
tail(data)

,Remiss,Cell,Smear,Infil,Li,Blast,Temp
22,0,-9,-93,-84,-6.0,1.591,1.020
23,1,1,-58,-58,1.0,-531.000,1.002
24,0,-95,-32,-32,1.6,-886.000,-988.000
25,1,1,-6,-6,1.7,-964.000,-99.000
26,1,1,-69,-69,-9.0,-398.000,-986.000
27,0,1,-73,-73,-7.0,-398.000,-986.000


In [4]:
#split data

set.seed(200)
train_obs <- floor (0.8*nrow (data))
train_ind<-sample(seq_len(nrow(data)),size=train_obs)
test = -train_ind
train_data<-data[train_ind,]
test_data<-data[-train_ind,]
testing_high <- data$Remiss[test]

In [5]:
#fit genaralized linear model
regmod<-glm(Remiss~.,data = train_data,family=binomial())
summary(regmod)


Call:
glm(formula = Remiss ~ ., family = binomial(), data = train_data)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.28718  -0.59603  -0.06141   0.50818   2.02014  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)
(Intercept) -7.581259   6.814658  -1.112    0.266
Cell         0.034501   0.024899   1.386    0.166
Smear        0.017240   0.153690   0.112    0.911
Infil       -0.114398   0.139973  -0.817    0.414
Li           0.523774   0.336609   1.556    0.120
Blast       -0.006348   0.005820  -1.091    0.275
Temp        -0.001145   0.002350  -0.487    0.626

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 26.734  on 20  degrees of freedom
Residual deviance: 15.965  on 14  degrees of freedom
AIC: 29.965

Number of Fisher Scoring iterations: 7


In [6]:
#predict
prob<- predict(regmod, test_data,type="response")
prob1<- data.frame(prob)

In [7]:
head(prob1)

,prob
2,7.497492e-02
9,1.511490e-02
11,2.815053e-04
17,8.647806e-05
19,2.481645e-04
20,5.677527e-01


In [8]:
#create confusion matrix
result=ifelse(prob>0.5,1,0)

library(caret)
cm=confusionMatrix(as.factor(testing_high),
                   as.factor(result),positive = '1')

Loading required package: lattice
Loading required package: ggplot2
Error: package or namespace load failed for 'ggplot2' in loadNamespace(j <- i[[1L]], c(lib.loc, .libPaths()), versionCheck = vI[[j]]):
 there is no package called 'pillar'


ERROR: Error: package 'ggplot2' could not be loaded


In [9]:
cm

function (x) 
2.54 * x
<bytecode: 0x0000000041d127c0>
<environment: namespace:grDevices>

In [10]:
#look at cross table
library(gmodels)
CrossTable(testing_high,result,prop.chisq = FALSE)

Warning message:
"package 'gmodels' was built under R version 3.6.3"


 
   Cell Contents
|-------------------------|
|                       N |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  6 

 
             | result 
testing_high |         0 |         1 | Row Total | 
-------------|-----------|-----------|-----------|
           0 |         4 |         0 |         4 | 
             |     1.000 |     0.000 |     0.667 | 
             |     0.800 |     0.000 |           | 
             |     0.667 |     0.000 |           | 
-------------|-----------|-----------|-----------|
           1 |         1 |         1 |         2 | 
             |     0.500 |     0.500 |     0.333 | 
             |     0.200 |     1.000 |           | 
             |     0.167 |     0.167 |           | 
-------------|-----------|-----------|-----------|
Column Total |         5 |         1 |         6 | 
             |     0.833 |     0.167 |           | 
-------------|-----------

In [11]:
#combine and write to local
final=cbind(testing_high,prob1)
write.csv(final,"Remission_Preds.csv")